## Import

In [1]:
# from pyspark.sql import SparkSession
import pandas as pd
import warnings
import sys
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, lead, to_date, lpad, min, max
from pyspark.sql.window import Window
sys.path.append('../src')
from data import run, load_and_concat_csv, drop_na_columns, process_data, add_y_label,lppub_column_names, lppub_column_classes
warnings.simplefilter(action='ignore', category=pd.errors.DtypeWarning)
print("Package Imported")

Package Imported


In [2]:
# file_path = '../data/raw/2016Q1.csv'
# new_directory = '../data/processed'

# df = run(file_path, new_directory)


In [3]:
# # Initialize Spark session
# spark = SparkSession.builder.appName("MortgageDelinquency").getOrCreate()

# # Load the dataset (replace 'path_to_your_data.csv' with the actual path to your dataset)
# df = spark.read.csv('../data/processed/2016Q1.csv', header=True, inferSchema=True)

# # Convert 'ACT_PERIOD' from MMYYYY integer format to timestamp
# df = df.withColumn('ACT_PERIOD', lpad(df['ACT_PERIOD'].cast('string'), 6, '0'))
# df = df.withColumn('ACT_PERIOD', to_date(df['ACT_PERIOD'].cast('string'), 'MMyyyy'))
# df = df.withColumn('default_status', col('default_status').cast('int'))

# # Define a window specification to partition by LOAN_ID and order by ACT_PERIOD
# window_spec = Window.partitionBy('LOAN_ID').orderBy('ACT_PERIOD')

# # Create a column 'next_8_quarters' to look ahead for the next 8 quarters
# # Use the lag function to check DLQ_STATUS for the next 8 quarters for each LOAN_ID
# df = df.withColumn(
#     'next_8_quarters_default',
#     when(col('default_status') >= 3, 1).otherwise(0)
# )

# # Use the window spec to look at the next 8 quarters' default status
# df = df.withColumn(
#     'next_8_quarters_default',
#     lead('next_8_quarters_default', 1).over(window_spec)
# )

# # Create the 'y_label' column based on next 8 quarters' default status
# df = df.withColumn('y_label', when(col('next_8_quarters_default') == 1, 1).otherwise(0))

# # Drop the intermediate 'next_8_quarters_default' column
# df = df.drop('next_8_quarters_default')

# # Show the results
# df.select('LOAN_ID', 'ACT_PERIOD', 'default_status', 'y_label').show()

# # Optionally, save the processed DataFrame to a new CSV or Parquet file
# df_single = df.coalesce(1)
# df_single.write.option("header", "true").csv('../data/processed/2016Q1_ylabel.csv')


In [4]:
def process_mortgage_data(input_path: str, output_path: str):
    # Initialize Spark session
    spark = SparkSession.builder.appName("MortgageDelinquency").getOrCreate()

    # Load the dataset
    df = spark.read.csv(input_path, header=True, inferSchema=True)

    # Convert 'ACT_PERIOD' from MMYYYY integer format to timestamp
    df = df.withColumn('ACT_PERIOD', lpad(df['ACT_PERIOD'].cast('string'), 6, '0'))
    df = df.withColumn('ACT_PERIOD', to_date(df['ACT_PERIOD'].cast('string'), 'MMyyyy'))
    df = df.withColumn('default_status', col('default_status').cast('int'))

    # Define a window specification to partition by LOAN_ID and order by ACT_PERIOD
    window_spec = Window.partitionBy('LOAN_ID').orderBy('ACT_PERIOD')

    # Create a column 'next_8_quarters' to look ahead for the next 8 quarters
    df = df.withColumn(
        'next_8_quarters_default',
        when(col('default_status') >= 3, 1).otherwise(0)
    )

    # Use the window spec to look at the next 8 quarters' default status
    df = df.withColumn(
        'next_8_quarters_default',
        lead('next_8_quarters_default', 1).over(window_spec)
    )

    # Create the 'y_label' column based on next 8 quarters' default status
    df = df.withColumn('y_label', when(col('next_8_quarters_default') == 1, 1).otherwise(0))

    # Drop the intermediate 'next_8_quarters_default' column
    df = df.drop('next_8_quarters_default')

    # Convert Spark DataFrame to pandas DataFrame and save to CSV
    df.coalesce(1).write.option("header", "true").csv(output_path)

    print(f"Data processed and saved to {output_path}")


In [5]:
# # Example usage:
# input_dirc = '../data/raw/'
# processed_dirc = '../data/processed/'
# output_dirc = '../data/cleaned/'
# file_name = '2015Q1.csv'
# df = run(input_dirc+file_name, processed_dirc)
# process_mortgage_data(processed_dirc + file_name, output_dirc + file_name)

In [6]:
# input_dirc = '../data/raw/'
# processed_dirc = '../data/processed/'
# output_dirc = '../data/cleaned/'

# # List of filenames from 2015Q2 to 2016Q4
# file_names = ['2015Q2.csv', '2015Q3.csv', '2015Q4.csv', 
#               '2016Q1.csv', '2016Q2.csv', '2016Q3.csv', '2016Q4.csv']

# # Loop through the list of files and process each one
# for file_name in file_names:
#     # Run for each file from raw to processed
#     df = run(input_dirc + file_name, processed_dirc)
    
#     # Process the mortgage data and save to cleaned directory
#     process_mortgage_data(processed_dirc + file_name, output_dirc + file_name)

#     print(f"Processing completed for {file_name}")

In [7]:
input_dirc = '../data/raw/'
processed_dirc = '../data/processed/'
output_dirc = '../data/cleaned/'

# List of filenames from 2015Q2 to 2016Q4
file_names = ['2016Q4.csv']

# Loop through the list of files and process each one
for file_name in file_names:
    # Run for each file from raw to processed
    df = run(input_dirc + file_name, processed_dirc)
    
    # Process the mortgage data and save to cleaned directory
    process_mortgage_data(processed_dirc + file_name, output_dirc + file_name)

    print(f"Processing completed for {file_name}")

Loading CSV data...
Processing data...
Saving processed data...
Processed data saved to: ../data/processed/2016Q4.csv


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/03 16:08:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
OpenJDK 64-Bit Server VM warning: CodeCache is full. Compiler has been disabled.
OpenJDK 64-Bit Server VM warning: Try increasing the code cache size using -XX:ReservedCodeCacheSize=


CodeCache: size=131072Kb used=22168Kb max_used=22364Kb free=108903Kb
 bounds [0x0000000106984000, 0x0000000107fa4000, 0x000000010e984000]
 total_blobs=9188 nmethods=8264 adapters=836
 compilation: enabled


Data processed and saved to ../data/cleaned/2016Q4.csv
Processing completed for 2016Q4.csv
